# Access NWIS with the USGS dataretrieval package

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mrahnis/nb-streamgage/blob/main/Streamgage-01--Access-NWIS-with-dataretrieval.ipynb)

## The USGS dataretrieval package

This package allows users to retrieve data using the USGS NWIS API. It is possible to get longer timeseries than is possible from the NWIS webpage. The dataretrieval git repository is here: https://github.com/USGS-python/dataretrieval


## Setup and imports

In [1]:
# if the notebook is running in colab we'll get the data from github
HOST_IS_COLAB = 'google.colab' in str(get_ipython())

if HOST_IS_COLAB:
    # if using the regular Colab runtime install dataretrieval and others packages
    !pip install dataretrieval --quiet --exists-action i
    !pip install pyproj --quiet --exists-action i
    !pip install xyzservices --quiet --exists-action i

In [2]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataretrieval import nwis

## Get the USGS sites in Lancaster County

In [3]:
# parameter codes for discharge and turbidity
parameterCd = ["00060", "63680"]
#parameterCd = ["00060"]
#parameterCd = ["63680"]

# get_info() accepts arguments: sites, stateCd as two letter postal code, huc-8, bBox in W,S,E,N decimal lat-lon pairs, and more recently countyCd as a FIPS code
# modifiedSince='YYYY-MM-DD' should give sites active since date
# countyCd='42071' should give Lancaster County PA
'''
sites, md = nwis.get_info(
    stateCd='PA',
    parameterCd=parameterCd,
    siteType='ST',
    startDt="2011-10-01",
    endDt=datetime.date.today().isoformat()
)



'''
frames = []
for code in parameterCd:    
    sites, md = nwis.get_info(
        stateCd='PA',
        parameterCd= code,
        siteType='ST',
        startDt="2011-10-01",
        endDt=datetime.date.today().isoformat()
    )
    
    sites['param'] = code
    frames.append(sites)
    
sites_codes = pd.concat(frames, axis=0)
sites_codes.groupby('site_no')



## Map the Sites

### Bokeh

In [4]:
import bokeh
from bokeh.models import ColumnDataSource, OpenURL, TapTool
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import xyzservices.providers as xyz
from pyproj import Transformer


def do_transform(lon, lat, transformer):
  return transformer.transform(lon, lat)

output_notebook()

WGS_TO_WEBMERCATOR = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)

x, y = do_transform(sites['dec_long_va'], sites['dec_lat_va'], WGS_TO_WEBMERCATOR)
sites['northing'] = y.tolist()
sites['easting'] = x.tolist()


# range bounds supplied in web mercator coordinates
collar = 5000

p = figure(
    x_range=(x.min()-collar, x.max()+collar),
    y_range=(y.min()-collar, y.max()+collar),
    x_axis_type="mercator",
    y_axis_type="mercator",
    tooltips = [
        ("name", "@station_nm"),
        ("number", "@site_no"),
        ("(Long, Lat)", "(@dec_long_va, @dec_lat_va)")
    ]
)

source = ColumnDataSource(sites)

if int(bokeh.__version__[0]) < 3:
    from bokeh.tile_providers import get_provider
    p.add_tile(get_provider('OSM'))
else:
    p.add_tile(xyz.OpenStreetMap.Mapnik)
print("Using Bokeh version {}".format(bokeh.__version__[0]))

p.scatter(
    x='easting', y='northing',
    size=10,
    fill_color='blue', fill_alpha=0.6,
    line_color=None,
    source=source
)

show(p)

Loading BokehJS ...

Using Bokeh version 3


## Get site information and statistics

In [5]:
gage = '015765195'
gage_info = sites[sites['site_no']==gage]
gage_info

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no,param,northing,easting
131,USGS,015765195,"Big Spring Run near Mylin Corners, PA",ST,395945.37,761550.54,39.995936,-76.264039,N,S,...,NaN,NaN,NaN,NaN,NaN,NaN,2476DFS,63680,4.865352e+06,-8.489674e+06


In [6]:
gage_stats, _ = nwis.get_stats(sites=gage)
gage_stats

,agency_cd,site_no,parameter_cd,ts_id,loc_web_ds,month_nu,day_nu,begin_yr,end_yr,count_nu,...,mean_va,p05_va,p10_va,p20_va,p25_va,p50_va,p75_va,p80_va,p90_va,p95_va
0,USGS,015765195,00010,170026,NaN,1,1,2013,2024,12,...,7.6,NaN,3.7,6.1,6.4,7.8,9.0,9.3,10.7,NaN
1,USGS,015765195,00010,170026,NaN,1,2,2013,2024,12,...,7.4,NaN,3.9,6.0,6.1,7.2,9.0,9.2,10.4,NaN
2,USGS,015765195,00010,170026,NaN,1,3,2013,2024,12,...,7.0,NaN,3.2,4.8,5.6,7.4,8.8,9.2,9.8,NaN
3,USGS,015765195,00010,170026,NaN,1,4,2013,2024,12,...,7.2,NaN,3.5,5.2,6.1,7.3,8.6,9.2,10.5,NaN
4,USGS,015765195,00010,170026,NaN,1,5,2013,2024,12,...,6.7,NaN,2.5,4.6,4.9,6.5,8.5,9.0,10.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,USGS,015765195,63680,214327,NaN,12,27,2017,2024,8,...,7.2,NaN,NaN,1.4,1.7,4.2,14.0,17.0,NaN,NaN
1460,USGS,015765195,63680,214327,NaN,12,28,2017,2024,8,...,12.0,NaN,NaN,1.4,1.7,4.2,24.0,27.0,NaN,NaN
1461,USGS,015765195,63680,214327,NaN,12,29,2017,2024,8,...,4.2,NaN,NaN,2.1,2.4,4.0,5.6,6.2,NaN,NaN
1462,USGS,015765195,63680,214327,NaN,12,30,2017,2024,7,...,4.8,NaN,NaN,1.8,2.0,3.1,7.6,9.6,NaN,NaN


## Get a gage record

In [7]:
start = '2021-01-01'
end = datetime.datetime.today().date()
service = 'iv' # daily value dv, or instantaneous value iv
df = nwis.get_record(sites=gage, service=service, start=start, end=end)
df.head()

,site_no,00010,00010_cd,00060,00060_cd,00065,00065_cd,00095,00095_cd,63680,63680_cd
datetime,,,,,,,,,,,
2021-01-01 05:00:00+00:00,015765195,7.6,A,2.92,A,3.60,A,661.0,A,2.0,A
2021-01-01 05:15:00+00:00,015765195,7.5,A,2.82,A,3.59,A,661.0,A,1.9,A
2021-01-01 05:30:00+00:00,015765195,7.5,A,2.82,A,3.59,A,661.0,A,2.1,A
2021-01-01 05:45:00+00:00,015765195,7.4,A,2.82,A,3.59,A,661.0,A,1.9,A
2021-01-01 06:00:00+00:00,015765195,7.4,A,2.82,A,3.59,A,661.0,A,1.9,A


Looking at `df` we will see it has several other codes. The NWIS codes included here stand for:
- 00010 : Temperature, water, degrees Celsius
- 00060 : Discharge, cubic feet per second
- 00065 : Gage height, feet
- 00095 : Specific conductance, water, unfiltered, microsiemens per centimeter at 25 degrees Celsius
- 63680 : Turbidity, water, unfiltered, monochrome near infra-red LED light, 780-900 nm, detection angle 90 +-2.5 degrees, formazin nephelometric units (FNU)

We can describe them to obtain some summary statistics. 

In [8]:
df.describe()

,00010,00060,00065,00095,63680
count,117580.000000,112967.000000,118317.000000,117484.000000,114608.000000
mean,12.189832,2.674700,3.738627,756.143313,4.486193
std,3.923330,6.351849,0.141008,122.336687,10.356642
min,0.600000,0.400000,3.350000,86.000000,0.300000
25%,8.800000,1.450000,3.680000,731.000000,1.500000
50%,12.300000,1.940000,3.740000,766.000000,2.400000
75%,15.400000,2.490000,3.790000,796.000000,4.000000
max,25.600000,260.000000,7.470000,3870.000000,405.000000


## Save as parquet

Saving a DataFrame in Parquet format has some advantages over saving to CSV. Parquet files tend to be smaller on disk and faster to read. Parquet will maintain your data types so you do not need to specify dtypes or parse datetime strings on re-reading the file.

In [9]:
df.to_parquet('nwis_{}_{}_{}.parquet'.format(gage, start, end), index=True)